In [ ]:
PORT = 5000
env = 'LOCAL' # LOCAL | ICL

In [ ]:
from pathlib import Path

if env == 'LOCAL':
    WORK_DIR = Path('.')
elif env == 'ICL':
    import os
    WORK_DIR = Path('/vol/bitbucket/') / os.getenv("USER") / 'worker'

In [ ]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI, WebSocket, WebSocketDisconnect, HTTPException, BackgroundTasks
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from src.constants import UI_URL, WORKER_URL
from src.models import ProcessRequest
from src.socket import Socket

import asyncio

nest_asyncio.apply()

app = FastAPI()

origins = [
    str(UI_URL),
    str(WORKER_URL),
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# TODO: If the UI never connects persist the result for sometime
# and return it when the UI tries to connect
async def process_work(data: dict):
    # Simulate work processing
    steps = 10
    for i in range(steps):
        await asyncio.sleep(1)
        await Socket.send({"complete": i, "total": steps})


@app.post("/process")
async def process(request: ProcessRequest, background_tasks: BackgroundTasks):
    Socket.register_incoming_id(request.socket)
    background_tasks.add_task(process_work, request)
    return JSONResponse(content={"status": 200})

@app.websocket("/ws/{socket_id}")
async def websocket_endpoint(websocket: WebSocket, socket_id: str):
    async with Socket(websocket, socket_id) as connection:
        await connection.listen()


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=PORT)